In [1]:
import os
import numpy as np
from PIL import Image
from ISR.models import RRDN

print ("Env : ", os.environ['CONDA_DEFAULT_ENV'])

datapath = '/media/xav/data/pday1/'
img = Image.open(datapath+'sample/téléchargement.jpeg')
lr_img = np.array(img.convert("RGB"))


Env :  projetday1


In [2]:
"""
import numpy as np
from PIL import Image

#img = Image.open('/home/xav/Images/Capture d’écran de 2020-08-17 19-21-21.png' )

img = Image.open(datapath+'sample/téléchargement.jpeg')
lr_img = np.array(img.convert("RGB"))
print(lr_img.shape)
#Image.fromarray(lr_img)
#img = Image.open(datapath+'sample/sam.png' )

#lr_img = np.array(img)
#print(lr_img.shape)

from ISR.models import RDN

rdn = RDN(arch_params={'C':6, 'D':20, 'G':64, 'G0':64, 'x':2})
rdn.model.load_weights(datapath+ 'weights/rdn-C6-D20-G64-G064-x2_ArtefactCancelling_epoch219.hdf5')
#print(np.concatenate((lr_img,)*2, axis=-1).shape)
#Image.fromarray(np.concatenate((lr_img,)*3, axis=-1))

#rdn = RDN(weights='psnr-small')
sr_img = rdn.predict(lr_img, by_patch_of_size=50)

#sr_img = rdn.predict(lr_img)
Image.fromarray(sr_img)

#sr_img = model.predict(image, by_patch_of_size=50)


"""


'\nimport numpy as np\nfrom PIL import Image\n\n#img = Image.open(\'/home/xav/Images/Capture d’écran de 2020-08-17 19-21-21.png\' )\n\nimg = Image.open(datapath+\'sample/téléchargement.jpeg\')\nlr_img = np.array(img.convert("RGB"))\nprint(lr_img.shape)\n#Image.fromarray(lr_img)\n#img = Image.open(datapath+\'sample/sam.png\' )\n\n#lr_img = np.array(img)\n#print(lr_img.shape)\n\nfrom ISR.models import RDN\n\nrdn = RDN(arch_params={\'C\':6, \'D\':20, \'G\':64, \'G0\':64, \'x\':2})\nrdn.model.load_weights(datapath+ \'weights/rdn-C6-D20-G64-G064-x2_ArtefactCancelling_epoch219.hdf5\')\n#print(np.concatenate((lr_img,)*2, axis=-1).shape)\n#Image.fromarray(np.concatenate((lr_img,)*3, axis=-1))\n\n#rdn = RDN(weights=\'psnr-small\')\nsr_img = rdn.predict(lr_img, by_patch_of_size=50)\n\n#sr_img = rdn.predict(lr_img)\nImage.fromarray(sr_img)\n\n#sr_img = model.predict(image, by_patch_of_size=50)\n\n\n'

In [3]:
"""
rdn = RDN(arch_params={'C': 3, 'D':10, 'G':64, 'G0':64, 'x':2})
rdn.model.load_weights(datapath+ 'weights/rdn-C3-D10-G64-G064-x2_PSNR_epoch134.hdf5')
sr_img = rdn.predict(lr_img, by_patch_of_size=50)
Image.fromarray(sr_img)
"""

"\nrdn = RDN(arch_params={'C': 3, 'D':10, 'G':64, 'G0':64, 'x':2})\nrdn.model.load_weights(datapath+ 'weights/rdn-C3-D10-G64-G064-x2_PSNR_epoch134.hdf5')\nsr_img = rdn.predict(lr_img, by_patch_of_size=50)\nImage.fromarray(sr_img)\n"

In [4]:
from ISR.models import RRDN
from ISR.models import Discriminator
from ISR.models import Cut_VGG19

lr_train_patch_size = 40
layers_to_extract = [5, 9]
scale = 2
hr_train_patch_size = lr_train_patch_size * scale

rrdn  = RRDN(arch_params={'C':4, 'D':3, 'G':64, 'G0':64, 'T':10, 'x':scale}, patch_size=lr_train_patch_size)
f_ext = Cut_VGG19(patch_size=hr_train_patch_size, layers_to_extract=layers_to_extract)
discr = Discriminator(patch_size=hr_train_patch_size, kernel_size=3)

In [5]:
from ISR.train import Trainer

loss_weights = {
  'generator': 0.0,
  'feature_extractor': 0.0833,
  'discriminator': 0.01
}
losses = {
  'generator': 'mae',
  'feature_extractor': 'mse',
  'discriminator': 'binary_crossentropy'
}

log_dirs = {'logs': './logs', 'weights': './weights'}

learning_rate = {'initial_value': 0.0004, 'decay_factor': 0.5, 'decay_frequency': 30}

flatness = {'min': 0.0, 'max': 0.15, 'increase': 0.01, 'increase_frequency': 5}

adam_optimizer = {'beta1': 0.9, 'beta2': 0.999, 'epsilon': None}

trainer = Trainer(
    generator=rrdn,
    discriminator=discr,
    feature_extractor=f_ext,
    lr_train_dir=datapath+'div2k/DIV2K_train_LR_bicubic/X2/',
    hr_train_dir=datapath+'div2k/DIV2K_train_HR/',
    lr_valid_dir=datapath+'div2k/DIV2K_train_LR_bicubic/X2/',
    hr_valid_dir=datapath+'div2k/DIV2K_train_HR/',
    loss_weights=loss_weights,
    losses=losses,
    learning_rate=learning_rate,
    flatness=flatness,
    log_dirs=log_dirs,
    adam_optimizer=adam_optimizer,
    metrics={'generator': 'PSNR_Y'},
    dataname='div2k',
    weights_generator=None,
    weights_discriminator=None,
    n_validation=40,
)


In [6]:
trainer.train(
    epochs=1,
    steps_per_epoch=20,
    batch_size=4,
    monitored_metrics={'val_generator_PSNR_Y': 'max'}
)



Training details:
  training_parameters: 
    lr_train_dir: /media/xav/data/pday1/div2k/DIV2K_train_LR_bicubic/X2/
    hr_train_dir: /media/xav/data/pday1/div2k/DIV2K_train_HR/
    lr_valid_dir: /media/xav/data/pday1/div2k/DIV2K_train_LR_bicubic/X2/
    hr_valid_dir: /media/xav/data/pday1/div2k/DIV2K_train_HR/
    loss_weights: {'generator': 0.0, 'feature_extractor': 0.0833, 'discriminator': 0.01}
    log_dirs: {'logs': './logs', 'weights': './weights'}
    fallback_save_every_n_epochs: 2
    dataname: div2k
    n_validation: 40
    flatness: {'min': 0.0, 'max': 0.15, 'increase': 0.01, 'increase_frequency': 5}
    learning_rate: {'initial_value': 0.0004, 'decay_factor': 0.5, 'decay_frequency': 30}
    adam_optimizer: {'beta1': 0.9, 'beta2': 0.999, 'epsilon': None}
    losses: {'generator': 'mae', 'feature_extractor': 'mse', 'discriminator': 'binary_crossentropy'}
    metrics: {'generator': <function PSNR_Y at 0x7f9ac42210e0>}
    lr_patch_size: 40
    steps_per_epoch: 20
    batch_siz

Epoch 0/1
Current learning rate: 0.00039999998989515007
100%|██████████| 20/20 [00:16<00:00,  1.22it/s]
Epoch 0 took       16.5s


160/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

{'val_loss': 0.789432785846293, 'val_generator_loss': 0.39677048, 'val_discriminator_loss': 0.0077741877, 'val_feature_extractor_loss': 3.359367, 'val_feature_extractor_1_loss': 15.592732, 'val_generator_PSNR_Y': 8.610157, 'train_d_real_loss': 0.062099487, 'train_d_real_accuracy': 1.0, 'train_d_fake_loss': 0.119073145, 'train_d_fake_accuracy': 0.99875, 'train_loss': 1.1911682, 'train_generator_loss': 0.42985448, 'train_discriminator_loss': 0.0056302827, 'train_feature_extractor_loss': 5.3516927, 'train_feature_extractor_1_loss': 23.246431, 'train_generator_PSNR_Y': 7.6335683}
val_generator_PSNR_Y improved from       -inf to    8.61016
Saving weights


In [7]:
#pred
!echo 3


3


In [8]:
#sr_img = rrdn.predict(lr_img)
sr_img = rrdn.predict(lr_img, by_patch_of_size=40)

Image.fromarray(sr_img)


ValueError: Error when checking input: expected LR_input to have shape (40, 40, 3) but got array with shape (44, 44, 3)